In [ ]:
!pip install stellargraph[demos]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 435 kB 4.2 MB/s 
     |████████████████████████████████| 500 kB 18.8 MB/s 
     |████████████████████████████████| 121 kB 21.5 MB/s 
     |████████████████████████████████| 1.6 MB 14.9 MB/s 
     |████████████████████████████████| 83 kB 754 kB/s 
     |████████████████████████████████| 41 kB 253 kB/s 
  Created wheel for mplleaflet: filename=mplleaflet-0.0.5-py3-none-any.whl size=28582 sha256=4c9ab8faf56bbf23a08309e9e0b1db9aa3c8f6c5e543e85ed1828409de84aa86
  Stored in directory: /root/.cache/pip/wheels/d4/a9/cf/8c4753b208759e9509ade040a94c3d4271f1756f529c575dc8
Successfully built mplleaflet


In [30]:
import json
import pandas as pd
import numpy as np
from sklearn import preprocessing, feature_extraction, model_selection
from sklearn.metrics import mean_absolute_error, mean_squared_error

import stellargraph as sg
from stellargraph.mapper import HinSAGELinkGenerator
from stellargraph.layer import HinSAGE, link_regression
from tensorflow.keras import Model, optimizers, losses, metrics

import multiprocessing
from stellargraph import datasets
from IPython.display import display, HTML
import matplotlib.pyplot as plt
%matplotlib inline

from google.colab import drive 
import sys


drive.mount('/content/drive', force_remount=True) 
colab_path="/content/drive/My Drive/csci2470/project"
sys.path.append(colab_path)

Mounted at /content/drive


In [31]:
batch_size = 200
epochs = 4
train_size = 0.7
test_size = 0.3
num_samples = [8, 4]
num_workers = 4

In [32]:
user_nodes = pd.read_csv(f"{colab_path}/nodes_user.csv")
subreddit_nodes = pd.read_csv(f"{colab_path}/nodes_subreddit.csv")
edges = pd.read_csv(f"{colab_path}/edges.csv")
edges = edges.drop(columns=['user_feat1', 'user_feat2', 'subreddit_feat1'])

pd.set_option('display.width', 100)

# print(user_nodes[:5])
# print(subreddit_nodes[:5])
# print(edges[:5])

# dataset = datasets.MovieLens()
# display(HTML(dataset.description))
# G, edges_with_ratings = dataset.load()
# print(G.info())

# user = pd.DataFrame(
#     {"duration": [2, 4, 6, 2, 4, 6, 2, 4, 6, 2, 4, 6, 2, 4, 6, 2, 4, 6, 2, 4]},
#     index = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20"]
# )

# subreddit = pd.DataFrame(
#     index = ["music", "drama", "game", "relation"]
# )

# join = pd.DataFrame(
#     {
#         "source": ["1", "1", "2", "3", "3", "4", "5", "6", "6", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20"], 
#         "target": ["music", "drama", "music", "drama", "game", "game", "music", "music", "drama", "game", "music", "drama", "game", "relation", "music", "drama", "game", "relation", "music", "drama", "game", "relation", "game", "relation"],
#         "freq": [4, 8, 2, 12, 10, 4, 2, 4, 6, 6, 2, 4, 6, 2, 4, 6, 2, 4, 6, 2, 4, 6, 7, 8]
#      }
# )
def u(users):
    return "u_" + users.astype(str)

def s(subreddits):
    return "s_" + subreddits.astype(str)

# user = pd.DataFrame(
#     {"user_feat1": user_nodes['user_feat1'], "user_feat2": user_nodes['user_feat2']},
#     index = user_nodes['username'],
# )

# subreddit = pd.DataFrame(
#     {"subreddit_feat1": subreddit_nodes['subreddit_feat1']},
#     index = subreddit_nodes['subreddit'],
# )

user_nodes['username'] = u(user_nodes['username'])
subreddit_nodes['subreddit'] = s(subreddit_nodes['subreddit'])
user_nodes.set_index("username", inplace=True)
subreddit_nodes.set_index("subreddit", inplace=True)

edges["username"] = u(edges["username"])
edges["subreddit"] = s(edges["subreddit"])

G = sg.StellarGraph(
    {"user": user_nodes, "subreddit": subreddit_nodes},
    {"freq": edges[["username", "subreddit"]]},
    source_column="username",
    target_column="subreddit",
  )

print(G.info())

StellarGraph: Undirected multigraph
 Nodes: 57577, Edges: 892462

 Node types:
  subreddit: [34967]
    Features: float32 vector, length 1
    Edge types: subreddit-freq->user
  user: [22610]
    Features: float32 vector, length 2
    Edge types: user-freq->subreddit

 Edge types:
    subreddit-freq->user: [892462]
        Weights: all 1 (default)
        Features: none


In [34]:
edges_train, edges_test = model_selection.train_test_split(
    edges, train_size=train_size, test_size=test_size
)

edgelist_train = list(edges_train[["username", "subreddit"]].itertuples(index=False))
edgelist_test = list(edges_test[["username", "subreddit"]].itertuples(index=False))

labels_train = edges_train["tot_comments"]
labels_test = edges_test["tot_comments"]


# edge_splitter_test = sg.data.EdgeSplitter(G)
# graph_test, edges_test, labels_test = edge_splitter_test.train_test_split(
#     p=0.2, method="global", edge_label="default"
# )

# edge_splitter_train = sg.data.EdgeSplitter(graph_test, G)
# graph_train, edges_train, labels_train = edge_splitter_train.train_test_split(
#     p=0.7, method="global", edge_label="default"
# )

generator = HinSAGELinkGenerator(
    G, batch_size, num_samples, head_node_types=["user", "subreddit"]
)

train_gen = generator.flow(edgelist_train, labels_train, shuffle=True)
test_gen = generator.flow(edgelist_test, labels_test)

generator.schema.type_adjacency_list(generator.head_node_types, len(num_samples))

[('user', [2]),
 ('subreddit', [3]),
 ('subreddit', [4]),
 ('user', [5]),
 ('user', []),
 ('subreddit', [])]

In [35]:
hinsage_layer_sizes = [32, 32]
assert len(hinsage_layer_sizes) == len(num_samples)

hinsage = HinSAGE(
    layer_sizes=hinsage_layer_sizes, generator=generator, bias=True, dropout=0.0
)
x_inp, x_out = hinsage.in_out_tensors()
score_prediction = link_regression(edge_embedding_method="concat")(x_out)

import tensorflow.keras.backend as K

def root_mean_square_error(s_true, s_pred):
    return K.sqrt(K.mean(K.pow(s_true - s_pred, 2)))


model = Model(inputs=x_inp, outputs=score_prediction)
model.compile(
    optimizer=optimizers.Adam(lr=1e-2),
    loss=losses.mean_squared_error,
    metrics=[root_mean_square_error, metrics.mae],
)

model.summary()

link_regression: using 'concat' method to combine node embeddings into edge embeddings
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 8, 1)]       0           []                               
                                                                                                  
 input_5 (InputLayer)           [(None, 32, 2)]      0           []                               
                                                                                                  
 input_6 (InputLayer)           [(None, 32, 1)]      0           []                               
                                                                                                  
 input_1 (InputLayer)           [(None, 1, 2)]       0           []                               
       

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
test_metrics = model.evaluate(
    test_gen, verbose=1, use_multiprocessing=False, workers=num_workers
)

print("Untrained model's Test Evaluation:")
for name, val in zip(model.metrics_names, test_metrics):
    print("\t{}: {:0.4f}".format(name, val))

history = model.fit(
    train_gen,
    epochs=epochs,
    verbose=1,
    shuffle=False,
    use_multiprocessing=False,
)

1339/1339 [==============================] - 243s 180ms/step - loss: 4351.0752 - root_mean_square_error: 61.4465 - mean_absolute_error: 15.8194
Untrained model's Test Evaluation:
	loss: 4351.0752
	root_mean_square_error: 61.4465
	mean_absolute_error: 15.8194
Epoch 1/4
 772/3124 [======>.......................] - ETA: 6:21 - loss: 3942.5190 - root_mean_square_error: 58.3381 - mean_absolute_error: 19.8892